In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
customer = pd.read_excel('../Finance/Raw datasets -DR DG-IIT D/Customers_31JAN2019.xlsx')
lms = []
for i in pd.read_csv('../Finance/Raw datasets -DR DG-IIT D/LMS_31JAN2019.csv', chunksize = 100000):
    lms.append(i)
    
lms = pd.concat(lms, ignore_index = True, sort = False)
train = pd.read_csv('../Finance/Raw datasets -DR DG-IIT D/train_foreclosure.csv')
test = pd.read_csv('../Finance/Raw datasets -DR DG-IIT D/test_foreclosure.csv')

FileNotFoundError: [Errno 2] File b'LMS_31JAN2019.csv' does not exist: b'LMS_31JAN2019.csv'

In [3]:
customer['CUSTOMERID'].nunique(), lms['CUSTOMERID'].nunique(), lms['AGREEMENTID'].nunique()

(10000, 32895, 33354)

In [4]:
train['AGREEMENTID'].nunique(), test['AGREEMENTID'].nunique()

(20012, 13342)

In [5]:
lms[lms['AGREEMENTID'] == 11220001]

,AGREEMENTID,CUSTOMERID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,...,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,MOB
0,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.734072,10.612692,58.0,120,1,...,40.06,39,19.0,0.0,0.6,HL,10901100.0,NaN,NaN,39
2855,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.734072,10.612692,58.0,120,1,...,40.06,40,18.0,0.0,0.6,HL,10901100.0,NaN,NaN,40
5825,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.421934,10.612692,58.0,120,1,...,40.06,41,17.0,0.0,0.6,HL,10901100.0,NaN,NaN,41
8928,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.421934,10.612692,58.0,120,1,...,40.06,41,17.0,0.0,0.6,HL,10901100.0,NaN,NaN,42
12193,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.421934,10.612692,58.0,120,1,...,40.06,43,15.0,0.0,0.6,HL,10901100.0,NaN,NaN,43
15534,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.421934,10.612692,58.0,120,1,...,40.06,44,14.0,0.0,0.6,HL,10901100.0,NaN,NaN,44
18931,11220001,12001000.0,11710107.24,11710107.24,2010-08-30,13.421934,10.612692,45.0,120,1,...,40.06,45,0.0,0.0,0.6,HL,10901100.0,NaN,NaN,45


In [5]:
# Merging train and test on aggregated LMS on AGREEMENTID 
# Only last record has been kept in aggregation
agg_lms = lms.groupby('AGREEMENTID').agg('last').reset_index()
train_lms = pd.merge(train, agg_lms, on = 'AGREEMENTID')
test_lms = pd.merge(test, agg_lms, on = 'AGREEMENTID')

In [6]:
train_lms.shape, test_lms.shape

((20012, 39), (13342, 39))

In [7]:
final_train = pd.merge(train_lms, customer, on = 'CUSTOMERID', how = 'left')
final_test = pd.merge(test_lms, customer, on = 'CUSTOMERID', how = 'left')

In [8]:
final_train.shape, final_test.shape

((20012, 53), (13342, 53))

In [9]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20012 entries, 0 to 20011
Data columns (total 53 columns):
AGREEMENTID              20012 non-null int64
FORECLOSURE              20012 non-null int64
CUSTOMERID               19731 non-null float64
LOAN_AMT                 20012 non-null float64
NET_DISBURSED_AMT        20012 non-null float64
INTEREST_START_DATE      20012 non-null object
CURRENT_ROI              20012 non-null float64
ORIGNAL_ROI              20012 non-null float64
CURRENT_TENOR            20012 non-null float64
ORIGNAL_TENOR            20012 non-null int64
DUEDAY                   20012 non-null int64
AUTHORIZATIONDATE        20012 non-null object
CITY                     20012 non-null object
PRE_EMI_DUEAMT           20012 non-null float64
PRE_EMI_RECEIVED_AMT     20012 non-null float64
PRE_EMI_OS_AMOUNT        20012 non-null float64
EMI_DUEAMT               20012 non-null float64
EMI_RECEIVED_AMT         20012 non-null float64
EMI_OS_AMOUNT            20012 non-nul

In [10]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13342 entries, 0 to 13341
Data columns (total 53 columns):
AGREEMENTID              13342 non-null int64
FORECLOSURE              0 non-null float64
CUSTOMERID               13164 non-null float64
LOAN_AMT                 13342 non-null float64
NET_DISBURSED_AMT        13342 non-null float64
INTEREST_START_DATE      13342 non-null object
CURRENT_ROI              13342 non-null float64
ORIGNAL_ROI              13342 non-null float64
CURRENT_TENOR            13342 non-null float64
ORIGNAL_TENOR            13342 non-null int64
DUEDAY                   13342 non-null int64
AUTHORIZATIONDATE        13342 non-null object
CITY                     13342 non-null object
PRE_EMI_DUEAMT           13342 non-null float64
PRE_EMI_RECEIVED_AMT     13342 non-null float64
PRE_EMI_OS_AMOUNT        13342 non-null float64
EMI_DUEAMT               13342 non-null float64
EMI_RECEIVED_AMT         13342 non-null float64
EMI_OS_AMOUNT            13342 non-null 

In [11]:
final_train.to_csv('../Finance/final_train.csv', index = False)
final_test.to_csv('../Finance/final_test.csv', index = False)

In [12]:
data = pd.read_csv('../Finance/data_last.csv')

In [13]:
print(data.shape)
data.head()

(33354, 51)


,AGREEMENTID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,AUTHORIZATIONDATE,...,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,OCCUPATION,POSITION,GROSS_INCOME,PRE_JOBYEARS,NETTAKEHOMEINCOME,BRANCH_PINCODE
0,11220001,11710107.24,11710107.24,30/08/10,13.421934,10.612692,45,120,1,29/08/10,...,M,M,POSTGRAD,0.0,NaN,NaN,198375.2218,8.0,198375.2218,400070.0
1,11220002,19290253.32,19290253.32,15/09/10,13.734072,10.612692,137,180,1,15/09/10,...,M,M,POSTGRAD,0.0,NaN,NaN,242703.9829,10.0,242703.9829,400070.0
2,11220003,4053498.66,4053498.66,06/09/10,17.167590,12.485520,360,180,5,06/09/10,...,M,M,GRAD,0.0,NaN,NaN,0.0000,NaN,0.0000,400070.0
3,11220004,13257031.92,13257031.92,29/09/10,15.606900,10.924830,198,180,5,29/09/10,...,M,M,GRAD,0.0,NaN,NaN,365263.5116,NaN,365263.5161,400070.0
4,11220005,13211403.04,13211403.04,06/09/10,16.543314,11.861244,142,180,5,06/09/10,...,M,M,GRAD,0.0,NaN,NaN,187342.1990,NaN,187342.1990,400070.0
